In [1]:
import requests,re,time
import pandas as pd

In [2]:
# using amap.com 's API to calculate the shortest driving time between city governments

key= '4cf31868da639ba20f04d33ca5218dff' #'6a803535ccaeb1ad3677a7ec2bf80c00'
locationdic={}
def getdic(url):
    count=0
    while True :
        try:
            res = requests.get(url)
            dic = res.json()

            return dic
        except:
            time.sleep(1)
            count+=1

In [3]:
def getxy(address):
    try:
        if address in locationdic:
            return locationdic[address]
        url=f'https://restapi.amap.com/v3/place/text?keywords={address}&offset=20&page=1&key={key}&citylimit=true'
        #res = requests.get(url)
        dic = getdic(url)
        pois=dic['pois']
        location=pois[0]['location']
        locationdic[address]=location
        return location
    except Exception as e:
        print(e)

In [4]:
def getdrive(s,e):

    url=f'https://restapi.amap.com/v3/direction/driving?origin={s}&destination={e}&key={key}'
    # res = requests.get(url)
    dic = getdic(url)
    route=dic['route']['paths']
    distance=float(route[0]['distance'])
    duration=float(route[0]['duration'])/60
    return  distance,duration

try:
    last_dis_df=pd.read_excel('distance.xlsx')
    last_dur_df = pd.read_excel('duration.xlsx')
    print('successfully read in the existing data')
except:
    print('no existing data')

successfully read in the existing data


In [5]:
places=['广州市越秀区府前路1号市政府大院', '深圳市民中心c区', '珠海市香洲区人民东路2号', '汕头市金平区跃进路28号', '佛山市禅城区岭南大道北12号', '韶关市浈江区风度北路75号', '河源市源城区沿江中路19号', '梅州市梅江区新中路38号', '惠州市惠城区云山西路6号', '汕尾市城区汕尾大道北市政府办公楼', '东莞市南城胜和鸿福路99号', '中山市松苑路1号', '江门市蓬江区白沙大道西1号', '阳江市江城区东风二路60号', '湛江市赤坎区跃进路67号', '茂名市茂南区油城六路2号', '肇庆市端州区城中路49号', '清远市清城区人民二路18号', '潮州市湘桥区枫春路1号', '揭阳市榕城区临江北路39号', '云浮市云城区世纪大道行政中心',
       '澳门特别行政区风顺堂区南湾湖景大马路131号','香港金钟添美道2号']
columns=[' ',]+places
distance_datas=[]
duration_datas=[]
for sindex,sname in enumerate(places):
    distance_oneline=[sname,]
    duration_oneline = [sname,]
    s = getxy(sname)
    for eindex,ename in enumerate(places):
        if ename==sname:
            distance_oneline.append(0)
            duration_oneline.append(0)
        else:

            e = getxy(ename)
            distance, duration = getdrive(s, e)
            try:
                distance=min(distance,last_dis_df.iloc[sindex,eindex+1])
                duration = min(duration, last_dur_df.iloc[sindex, eindex + 1])
            except:
                pass
            distance_oneline.append(distance)
            duration_oneline.append(duration)
    distance_datas.append(distance_oneline)
    duration_datas.append(duration_oneline)
    print(distance_oneline)
    print(duration_oneline)

['广州市越秀区府前路1号市政府大院', 0, 136413.0, 127163.0, 416688, 34164.0, 220999.0, 199319.0, 385989.0, 141655.0, 270929.0, 69724.0, 84531.0, 90632.0, 224934, 421180.0, 340959.0, 98409, 77209.0, 410403, 379232, 145008.0, 139152.0, 167384.0]
['广州市越秀区府前路1号市政府大院', 0, 107.8833333333333, 118.8166666666667, 290.4166666666667, 40.13333333333333, 166.3, 147.25, 265.3666666666667, 126.8, 200.5166666666667, 65.9, 90.65, 82.31666666666666, 167.8833333333333, 304.35, 244.2, 89.76666666666667, 72.78333333333333, 293.35, 267.2333333333333, 112.95, 128.4833333333333, 145.5666666666667]
['深圳市民中心c区', 135288.0, 0, 154287.0, 332854, 136652.0, 329192.0, 176358, 342374.0, 92215, 168095.0, 74346.0, 118236, 145557.0, 303278, 499523.0, 419302.0, 217884.0, 199342.0, 356248, 325077, 264482.0, 164246.0, 65777.0]
['深圳市民中心c区', 106.8333333333333, 0, 132.4166666666667, 251.6666666666667, 114.7166666666667, 241.75, 139.65, 237.7, 88.96666666666667, 137.0333333333333, 70.31666666666666, 103.9166666666667, 119.1, 219.95, 352.583333

In [6]:
dis_df=pd.DataFrame(distance_datas,columns=columns)
dur_df=pd.DataFrame(duration_datas,columns=columns)
dis_df.to_excel('distance.xlsx',index=None)
dur_df.to_excel('duration.xlsx',index=None)